In [1]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
import os
# os.environ["OPENAI_API_KEY"] = "sk-proj-myapikey"
# os.environ["EXCHANGERATE_API_KEY"] = "YOUR-API-KEY"
exapi = os.environ["EXCHANGERATE_API_KEY"]

In [3]:
# we will hit https://www.exchangerate-api.com/docs/pair-conversion-requests api and get back result
# then integrate result to produce real-time output from LLM

# 1. fetch conversion factor (ex 84.39)
# 2. multiply conversion factor with input dollar and return back the result


In [4]:
# tool create
# exchange tool
from typing import Annotated
from langchain_core.tools import InjectedToolArg


@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between two currencies.
    """
    return requests.get(f"https://v6.exchangerate-api.com/v6/{exapi}/pair/{base_currency}/{target_currency}").json()["conversion_rate"]


# multiplication tool
@tool
# def convert(base_currency_value: float, coversion_factor: float) -> float:    # concrete but we should pass in runtime 
def convert(base_currency_value: float, conversion_factor: Annotated[float, InjectedToolArg]) -> float:
    """
    From a given base currency value and conversion factor, this function returns the converted target currency value.
    """
    return base_currency_value * conversion_factor

In [5]:
# get_conversion_factor("USD", "INR") # not work as it's not normal function anymore
cf = get_conversion_factor.invoke({"base_currency": "USD", "target_currency": "INR"})
cf

86.2207

In [6]:
convert.invoke({"base_currency_value":10, "conversion_factor":cf})

862.2069999999999

In [7]:
# tool building
llm = ChatOpenAI()

In [8]:
llm_with_tools = llm.bind_tools( [get_conversion_factor,convert] ) # bind_function() -> depricated

In [9]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 500 usd to inr')]


In [10]:
ai_msg = llm_with_tools.invoke(messages)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PXsPrt1ch6ZgnrMVEFFvAT6M', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_Ypd14YIcA4zdWmiqcgjhLyHu', 'function': {'arguments': '{"base_currency_value": 500}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 117, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvOP8BVuKxSqozcuJPsnHVIaNhisP', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--509d4b26-b025-4349-bc5d-96804513bfb5-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_cu

In [11]:
messages.append(ai_msg)

In [12]:
ai_msg.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_PXsPrt1ch6ZgnrMVEFFvAT6M',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 500},
  'id': 'call_Ypd14YIcA4zdWmiqcgjhLyHu',
  'type': 'tool_call'}]

In [13]:
import json

for tool_call in ai_msg.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch this conversion rate
        conversion_factor = tool_message1.content
        # append this tool message to messages list
        messages.append(tool_message1)
    # execute the 2nd tool using the conversion rate from tool 1 # what if this is tool_calls[0] ?
    if tool_call['name'] == 'convert':
        # fetch the current arg
        tool_call['args']['conversion_factor'] = conversion_factor
        print(tool_call)
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)
print(messages)

{'name': 'convert', 'args': {'base_currency_value': 500, 'conversion_factor': '86.2207'}, 'id': 'call_Ypd14YIcA4zdWmiqcgjhLyHu', 'type': 'tool_call'}
[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 500 usd to inr', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PXsPrt1ch6ZgnrMVEFFvAT6M', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_Ypd14YIcA4zdWmiqcgjhLyHu', 'function': {'arguments': '{"base_currency_value": 500}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 117, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cac

In [14]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 500 usd to inr', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PXsPrt1ch6ZgnrMVEFFvAT6M', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_Ypd14YIcA4zdWmiqcgjhLyHu', 'function': {'arguments': '{"base_currency_value": 500}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 117, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvOP8BVuKxSqozcuJPsnHVIaNhisP', 'service_tier': 'de

In [15]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is approximately 86.2207. \n\nTherefore, converting 500 USD to INR results in 43,110.35 INR.'

**Was above ai-agent?**

ANS: Not exactly. ai-agent is fully autonomus. it can breakdown problem and solve step-by-step. no need of help in-between. 
But, here we made most of the code and decision making.

In [16]:
from langchain.agents import initialize_agent, AgentType

agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # (prompt) using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\sk\AppData\Local\Temp\ipykernel_1332\1000534065.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [17]:
user_query = "what's 10 usd in inr?"

# run the agent
response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: This requires converting USD to INR using the currency conversion factor.
Action:
```{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "INR"
  }
}```
Observation: 86.2207
Thought:Action:
```
{
  "action": "convert",
  "action_input": {
    "base_currency_value": 10,
    "conversion_factor": 86.2207
  }
}
```
Observation: 862.2069999999999
Thought:{
  "action": "Final Answer",
  "action_input": "10 USD is approximately 862.21 INR"
}

> Finished chain.
